In [86]:
import numpy as np
import pandas as pd
import imblearn                                      
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer


In [87]:
final_df = pd.read_csv('initial_preprocessing.csv')

In [88]:
var_test_datapoints = round(final_df.shape[0] * 0.1)
df_nans = final_df[final_df['3MResponse'] == 'Unknown']          ### making a new dataframe for unknown values in the target variable
final_df = final_df[final_df['3MResponse'] != 'Unknown']         ### Removing the unknown category 
model_label = LabelEncoder()
final_df['3MResponse'] = model_label.fit_transform(final_df['3MResponse'].astype('str'))    ### encoding the target variable


In [89]:
df_nans_x = df_nans.drop('3MResponse',1)            ### separting x and y for the target variables having unknown
df_nans_y = df_nans['3MResponse']

<ipython-input-89-c22e693e6c25>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_nans_x = df_nans.drop('3MResponse',1)            ### separting x and y for the target variables having unknown


In [90]:
rs = 1337
X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = final_df['3MResponse']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=var_test_datapoints,random_state = rs)
combined_train_df_x = pd.concat([X_train, df_nans_x])

<ipython-input-90-4ae165d2306b>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [91]:
from sklearn.impute import KNNImputer                         ### Imputing the null values using the X(features) values for both known and unknown values for the target variable
imputer = KNNImputer(n_neighbors=15)
fit_train = imputer.fit(combined_train_df_x)                  # (shape = 250,77) does not include target variable
imput_train = fit_train.transform(combined_train_df_x)     
c = list(combined_train_df_x.columns)
imput_train_df = pd.DataFrame(imput_train, columns = c)

imput_test = fit_train.transform(X_test)           ### Imputing the test dataset X(features) using the imputer trained from the train dataset
c = list(X_test.columns)
imput_test_df = pd.DataFrame(imput_test, columns = c)

df_nans_y = df_nans_y.replace('Unknown', np.nan)            ### combining X and y for the training set
combined_train_df_y = pd.concat([Y_train, df_nans_y],ignore_index=True)
imput_train_df['3MResponse'] = combined_train_df_y

fit_train_x_y = imputer.fit(imput_train_df)        ### Imputing the unknown values in the target variable using the combined X and y of the training set
imput_train_2 = fit_train_x_y.transform(imput_train_df)
c = list(imput_train_df.columns)
imput_train_df_x_y = pd.DataFrame(imput_train_2, columns = c)

In [92]:
for i in range(len(imput_train_df_x_y['3MResponse'])):             ### converting the target variable into integers
    if imput_train_df_x_y['3MResponse'][i] <= 0.5: 
        imput_train_df_x_y['3MResponse'][i] = 0
    elif imput_train_df_x_y['3MResponse'][i] >0.5 and imput_train_df_x_y['3MResponse'][i]<=1.5:
        imput_train_df_x_y['3MResponse'][i] = 1
    else:
        imput_train_df_x_y['3MResponse'][i] = 2

In [93]:
X_unknown = imput_train_df_x_y.drop('3MResponse',1)                            ### Dividing the dataframe into X and y and including the unknown datapoints
y_unknown = imput_train_df_x_y['3MResponse']

<ipython-input-93-b9bc94796b0b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = imput_train_df_x_y.drop('3MResponse',1)                            ### Dividing the dataframe into X and y and including the unknown datapoints


In [94]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in X_unknown.columns:
    if i not in labels:
        mean = X_unknown[i].mean()
        std = X_unknown[i].std()
        X_unknown[i] = (X_unknown[i] - mean) / std
        imput_test_df[i] = (imput_test_df[i] - mean) / std

In [95]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL','hxtia']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    X_unknown = X_unknown.drop(i,1)
    imput_test_df = imput_test_df.drop(i,1)

<ipython-input-95-dd1de88e422f>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = X_unknown.drop(i,1)
<ipython-input-95-dd1de88e422f>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_test_df = imput_test_df.drop(i,1)


In [96]:
def sampling(over_under,X,y):                                                ### Over sampling the dataset using smote method
    if over_under == 'under':
        sampler = imblearn.under_sampling.RandomUnderSampler()
        X_under, y_under = sampler.fit_resample(X, y)
        return X_under,y_under
    elif over_under == 'over':
        ros = RandomOverSampler(random_state=rs)
        X_over, y_over = ros.fit_resample(X, y)
        return X_over,y_over
    else:
        sampler = SMOTE(random_state = rs)
        X_smote, y_smote = sampler.fit_resample(X, y)
        return X_smote,y_smote
X_train_final,y_train_final = sampling('smote',X_unknown,y_unknown)

In [98]:
X_train_final['y'] = y_train_final
X_train_final.to_csv('train_method3_cf.csv', index = False)

In [99]:
Y_test1 = Y_test.reset_index()
Y_test1 = Y_test1.drop(columns=['index'])
imput_test_df['y'] = Y_test1
imput_test_df.to_csv('test_method3_cf.csv', index = False)